# Analyze Project Results


This notebook is indended to analyze and visualize the the accuracy of the FMC models in an interactive setting.

Note: some outputs automatically generated in `report_materials.py`, this notebook is meant to compliment that script and provide more granular control

## Setup

In [ ]:
import os
import os.path as osp
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import sys
import h5py
import re
from dateutil.relativedelta import relativedelta
sys.path.append('../src')
from utils import time_range, Dict, read_yml, read_pkl, print_dict_summary, str2time
from viz import plot_styles, plot_one, make_st_map_interactive

In [ ]:
ml_forecast_dir = "../outputs/forecast_outputs"

### Read Results


In [ ]:
overall = pd.read_csv(osp.join(ml_forecast_dir, "overall.csv"))
by_dt = pd.read_csv(osp.join(ml_forecast_dir, "by_dt.csv"))
by_hod = pd.read_csv(osp.join(ml_forecast_dir, "by_hod.csv"))
by_st = pd.read_csv(osp.join(ml_forecast_dir, "by_stid.csv"))
sts = pd.read_csv(osp.join(ml_forecast_dir, "stid_locs.csv"))
rnn = pd.read_csv(osp.join(ml_forecast_dir, "rnn_preds.csv"))
var = pd.read_csv(osp.join(ml_forecast_dir, "all_variables_summary.csv"))

by_st = by_st.merge(sts, on="stid", how="left")

In [ ]:
# ml_data = read_pkl(osp.join(ml_forecast_dir, "ml_data.pkl"))

## Accuracy by Low-Med-High

In [ ]:
bins = [0, 10, 20, float('inf')]
labels = ['Low (0-10)', 'Med (10-20)', 'High (20+)']

rnn['fm_label'] = pd.cut(rnn['fm'], bins=bins, labels=labels, right=False)

In [ ]:
table = (
    rnn.groupby('fm_label')
      .agg(Mean_Bias=('residual', 'mean'),
           RMSE=('squared_error', lambda x: np.sqrt(x.mean())),
           RelError=('abs_error', lambda x: x.mean() / rnn.loc[x.index, 'fm'].mean()))
      .reset_index()
)

table.columns = ["FMC Level", "Bias", "RMSE", "Relative Error"]

In [ ]:
table

## Loss Graphic

In [ ]:
start = pd.to_datetime(str2time("2024-01-01T00:00:00Z"))
end = pd.to_datetime(str2time("2024-01-02T23:00:00Z"))
st = "TT562"
rnn.date_time = pd.to_datetime(rnn.date_time)
df = rnn[(rnn.stid == st) & (rnn.date_time >= start) & (rnn.date_time <= end)]
df = df[df.rep == df.rep.unique()[0]]

In [ ]:
import matplotlib.dates as mdates
plt.plot(df.date_time, df.preds, color="k", linestyle="dashed", label="Predicted")
plt.plot(df.date_time, df.fm, **plot_styles["fm"])
plt.legend(loc="upper right")
plt.grid()
plt.ylim(7.5, 20)
plt.ylabel("FMC (%)")
plt.rcParams.update({"axes.labelsize": 14, "xtick.labelsize": 12, "ytick.labelsize": 12, "legend.fontsize": 12})
ax = plt.gca()
ax.xaxis.set_major_locator(mdates.HourLocator(interval=1))
ax.set_xticks(df.date_time[[0, 12, 24, 36, 47]])
ax.set_xticklabels(df.date_time[[0, 12, 24, 36, 47]].dt.strftime("%Y-%m-%d\n%H:%M"), rotation=45)
plt.tight_layout()
plt.savefig("../outputs/loss.png", dpi=600)

## Error by Location

In [ ]:
sts.shape

In [ ]:
# All locations
# make_st_map_interactive(sts)

In [ ]:
np.random.seed(20250509) # date of first run
sts["train"] = np.random.choice([1, 0], size=len(sts), p=[0.8, 0.2])

make_st_map_interactive(sts, color="train", binary=True)

In [ ]:
df = by_st[by_st.Model == "rnn"].copy()
df["MSE"] = df.loc[:,"mse_mean"]
# make_st_map_interactive(df, color="MSE")

In [ ]:
from sklearn.metrics import r2_score

df = by_st[by_st.Model=="rnn"]
x = df['elev'].values
y = df['mse_mean'].values
plt.figure(figsize=(8, 6))
plt.scatter(x, y, marker='o')

# # Fit linear trend line
# coeffs = np.polyfit(x, y, deg=1)
# trendline = np.poly1d(coeffs)
# y_pred = trendline(x)
# r2 = r2_score(y, y_pred)
# plt.plot(np.sort(x), trendline(np.sort(x)), color='red', label='Trend line')
# x_mid = np.median(x)
# y_mid = trendline(x_mid)
# plt.text(x_mid, y_mid, f'$R^2$ = {r2:.3f}', color='red', fontsize=12, verticalalignment='bottom')


plt.xlabel('Elevation')
plt.ylabel('Mean Squared Error (MSE)')
plt.title('MSE vs Elevation (RNN)')
plt.grid(True)
plt.tight_layout()
plt.show()

## RNN Errors

In [ ]:
# df2 = rnn[rnn.rep == 1]

fig, axs = plt.subplots(1, 2, figsize=(12, 6))
# Residual Plot
axs[0].scatter(rnn.preds, rnn.residual, marker="o", alpha=.7)
axs[0].set_xlabel("Predicted FMC (%)")
axs[0].set_ylabel("Residual (Observed - Predicted)")
axs[0].grid(True)
axs[0].axhline(y=0, linestyle="dashed", color="k")

# Residual Histogram
axs[1].hist(rnn.residual, bins=20, edgecolor="k")
axs[1].set_xlabel("Residual (Observed - Predicted)")
axs[1].set_ylabel("Frequency")
axs[1].grid(True)

plt.tight_layout()
plt.savefig(osp.join(ml_forecast_dir, "residuals.png"))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
# Observed Histogram
axs[0].hist(rnn.fm, bins=42, color="#468a29", edgecolor="k")
axs[0].set_xlabel("Observed FMC (%)")
axs[0].set_ylabel("Frequency")
axs[0].grid(True)
axs[0].set_xlim(0,40)

# Predicted Histogram
axs[1].hist(rnn.preds, bins=42, edgecolor="k")
axs[1].set_xlabel("Predicted FMC (%)")
axs[1].set_ylabel("Frequency")
axs[1].grid(True)
axs[1].set_xlim(0,40)
plt.tight_layout()
plt.savefig(osp.join(ml_forecast_dir, "fm_hist.png"))

In [ ]:
# Stratefying by 
# Low (0-10) Medium (10-20) High (20-30) Very High (30+)
bins = [0, 10, 20, float('inf')]
labels = ['Low (0-10)', 'Medium (10-20)', 'High (20+)']
rnn["fm_level"] = pd.cut(rnn["fm"], bins=bins, labels=labels, right=False)
bias = rnn.groupby(["rep", "fm_level"], observed=True)["residual"].agg("mean")
summary = pd.merge(
    bias.groupby("fm_level", observed=True).mean(),
    bias.groupby("fm_level", observed=True).std(),
    on="fm_level"
).reset_index()
summary.columns = ["FMC Level", "Bias", "Std"]
summary

Plotting some good and bad examples based on by station error.

In [ ]:
st = "CPPC2"
start = pd.to_datetime('2024-09-19 00:00:00+00:00')
end = start + relativedelta(hours=48-1)

df2 = rnn[rnn.stid == st]
df2.loc[:,"date_time"] = pd.to_datetime(df2.date_time)
df2 = df2[(df2.date_time >= start) & (df2.date_time <= end)]

mse = df2.groupby("rep")["squared_error"].mean()
print(f"MSE for {st}:")
print(f"    {mse.mean()}, ({mse.min()} to {mse.max()})")

In [ ]:
mean_pred = df2.groupby(["date_time"]).preds.mean()
std = df2.groupby(["date_time"]).preds.std()
high = mean_pred + std
low = mean_pred - std
x = df2.date_time.unique()
plot_one(ml_data, st=st, m=mean_pred, start_time = start, end_time=end, title2=f"MSE: {mse.mean().round(2)}")
plt.fill_between(x, low, high, color="k", alpha=0.2)

In [ ]:
st = "TT562"
start = pd.to_datetime('2024-01-01 00:00:00+00:00')
end = start + relativedelta(hours=48-1)

df2 = rnn[rnn.stid == st]
df2.loc[:,"date_time"] = pd.to_datetime(df2.date_time)
df2 = df2[(df2.date_time >= start) & (df2.date_time <= end)]

mse = df2.groupby("rep")["squared_error"].mean()
print(f"MSE for {st}:")
print(f"    {mse.mean()}, ({mse.min()} to {mse.max()})")

In [ ]:
mean_pred = df2.groupby(["date_time"]).preds.mean()
std = df2.groupby(["date_time"]).preds.std()
high = mean_pred + std
low = mean_pred - std
x = df2.date_time.unique()
plot_one(ml_data, st=st, m=mean_pred, start_time = start, end_time=end, title2=f"MSE: {mse.mean().round(2)}")
plt.fill_between(x, low, high, color="k", alpha=0.2, label=rf"$\pm$ 1 std. ({df2.rep.unique().shape[0]} reps)")
plt.legend(loc='upper left', bbox_to_anchor=(1, 0.5))

---

In [ ]:
st = "RLAS2"
start = pd.to_datetime('2024-02-08 00:00:00+00:00')
end = start + relativedelta(hours=48-1)

df2 = rnn[rnn.stid == st]
df2.loc[:,"date_time"] = pd.to_datetime(df2.date_time)
df2 = df2[(df2.date_time >= start) & (df2.date_time <= end)]

mse = df2.groupby("rep")["squared_error"].mean()
print(f"MSE for {st}:")
print(f"    {mse.mean()}, ({mse.min()} to {mse.max()})")

In [ ]:
mean_pred = df2.groupby(["date_time"]).preds.mean()
std = df2.groupby(["date_time"]).preds.std()
high = mean_pred + std
low = mean_pred - std
x = df2.date_time.unique()
plot_one(ml_data, st=st, m=mean_pred, start_time = start, end_time=end, title2=f"MSE: {mse.mean().round(2)}")
plt.fill_between(x, low, high, color="k", alpha=0.2, label=rf"$\pm$ 1 std. ({df2.rep.unique().shape[0]} reps)")
plt.legend(loc='upper left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.savefig(osp.join(ml_forecast_dir, "RLAS2_Feb824.png"))

In [ ]:
fm = df2[df2.rep == 23].fm
delta0 = fm.iloc[0] - mean_pred.iloc[0]
print(f"MSE for mean prediction: {mean_squared_error(mean_pred, fm)}")
print(f"Error at t=0: {delta0}")
print(f"MSE for translated mean prediction: {mean_squared_error(mean_pred+delta0, fm)}")

---

In [ ]:
st = "C3SKI"
start = pd.to_datetime('2024-10-19 00:00:00+00:00')
end = start + relativedelta(hours=48-1)

df2 = rnn[rnn.stid == st]
df2.loc[:,"date_time"] = pd.to_datetime(df2.date_time)
df2 = df2[(df2.date_time >= start) & (df2.date_time <= end)]

mse = df2.groupby("rep")["squared_error"].mean()
print(f"MSE for {st}:")
print(f"    {mse.mean()}, ({mse.min()} to {mse.max()})")

In [ ]:
mean_pred = df2.groupby(["date_time"]).preds.mean()
std = df2.groupby(["date_time"]).preds.std()
high = mean_pred + std
low = mean_pred - std

plot_one(ml_data, st=st, m=mean_pred, start_time = start, end_time=end, title2=f"MSE: {mse.mean().round(2)}")
x = df2.date_time.unique()
plt.fill_between(x, low, high, color="k", alpha=0.2, label=rf"$\pm$ 1 std. ({df2.rep.unique().shape[0]} reps)")
plt.legend(loc='upper left', bbox_to_anchor=(1, 0.5))

## TS Plots for CV Graphic

In [ ]:
start = pd.to_datetime('2023-01-01 00:00:00+00:00')
end = pd.to_datetime('2024-12-31 23:00:00+00:00')
x = time_range(start, end)
h2 = pd.to_datetime('2024-01-01 00:00:00+00:00')

In [ ]:
st_list = ["CHRC2", "TT689"]

In [ ]:
def p(st):
    df2 = ml_data[st]["data"]
    df2.loc[:,"date_time"] = pd.to_datetime(df2.date_time); df2 = df2.sort_values("date_time")
    plt.figure(figsize=(12,4))
    plt.grid()
    plt.xticks(fontsize=16, rotation=90)
    plt.yticks(fontsize=16) 
    plt.ylim(0,32)
    plt.ylabel("FMC (%)", fontsize=16)
    plt.plot(df2.date_time, df2.fm, color="#468a29")
    plt.tight_layout()
    plt.savefig(osp.join(ml_forecast_dir, f"{st}.png"))

for st in st_list:
    p(st)

In [ ]:
# st = "CHRC2"
# plot_one(ml_data, st=st, start_time = start, end_time=end, features=False)

In [ ]:
# st = "TT689"
# plot_one(ml_data, st=st, start_time = start, end_time=end, features=False)

In [ ]:
# st = "LKGC2"
# plot_one(ml_data, st=st, start_time = start, end_time=end, features=False)

In [ ]:
# st = "KRNK1"
# plot_one(ml_data, st=st, start_time = start, end_time=end, features=False)

## Errors over Year

Aggregate by month

In [ ]:
df = by_dt
df["date_time"] = pd.to_datetime(df.date_time)
df["month"] = df.date_time.dt.month
df = df.groupby(["Model", "month"]).mean().reset_index()

In [ ]:
# Plot each model
plt.figure()
for model in df["Model"].unique():
    subset = df[df["Model"] == model]
    plt.plot(subset["month"], subset["mse_mean"], label=model)

plt.xlabel("Month")
plt.ylabel("MSE Mean")
plt.title("MSE Mean by Month and Model")
plt.legend()
plt.grid(True)
plt.show()

## Reps

In [ ]:
rnn.groupby("stid")["rep"].nunique().mean()

In [ ]:
rnn.stid.unique().shape

In [ ]:
# total individuals
N = 151

# sample size (10% of N, rounded down)
sample_size = int(np.ceil((0.10 * N)))  # or use math.floor if you want explicit rounding

# number of draws
draws = 500

# probability of selecting a given individual in one sample
p = sample_size / N

# expected number of times one individual is selected
expected_times = draws * p

print(f"Expected number of times one individual is selected: {expected_times:.2f}")

In [ ]:
rnn.groupby("rep")["stid"].nunique()

In [ ]:
df = by_dt[by_dt.Model == "rnn"]
df.loc[:,"date_time"] = pd.to_datetime(df.date_time)
df = df.sort_values("date_time")

In [ ]:
from utils import time_range
tt = pd.to_datetime(time_range(start = "2024-01-01T00:00:00Z", end = "2024-12-31T23:00:00Z"))

In [ ]:
plt.scatter(df.date_time, df.mse_mean)